# GAOZのハミング距離最小でElection方式 (Election Gotoh)

### メモ  
GAOZで完全一致しないやつがある  
->ハミング距離最小のやつで絞る

### 担当者と日付
2024/10/☆☆ WHO

## 準備

### ライブラリのインポート  

In [151]:
import pandas as pd
import numpy as np
import warnings #ヘッダとかのやつが邪魔だったのでファイルにまとめてインポートしてる
warnings.simplefilter(action='ignore', category=FutureWarning)

In [152]:
# fmt: off
ALL_HEADER = ['Name', 'Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '247', '260', '653', '673', '810', '885',  # noqa: E501
                      '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017',  # noqa: E501
                      '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253',  # noqa: E501
                      '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489',  # noqa: E501
                      '3877', '3889'] # noqa

B_HEADERS_LIST = [
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '1920', '2017', '2087'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

B_USER_ATTRIBUTE_HEADERS = ['Gender', 'Age', 'Occupation', 'ZIP-code']

B_REVIEW_HEADERS_LIST = [
  ['260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['247', '1920', '2017', '2087'],  # noqa: E501
  ['260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

MOVIE_IDS = ['2', '56', '247', '260', '653', '673', '810', '885', '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253', '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489', '3877', '3889']
# fmt: on

### C_0 ~ C_9までの読み込み

In [153]:
from os import listdir

ATTACK_TARGET = "20"

#c0~c9に対するdfのリスト
c_data_list = []

files = listdir("../data")

for i in range(10):
    file = None
    for f in files:
        if f.startswith(f"C{ATTACK_TARGET}_{i}"):
            file = f
            break
    if file is None:
        print(f)
        raise("File not found")
    c_data = pd.read_csv(f"../data/C{ATTACK_TARGET}_{i}.csv")
    c_data_list.append(c_data)

c_data_list[0]

,Gender,Age,Occupation,ZIP-code,260,653,1525,2105,2193,2253,2628,2872,3438,3439,3440,3877,3889
0,M,35,15,6,4,5,4,2,2,3,1,0,0,2,3,0,0
1,F,18,9,6,0,0,1,4,5,3,4,0,0,1,0,0,4
2,M,18,10,6,0,0,0,5,4,5,3,0,1,0,2,0,5
3,F,45,7,6,1,3,1,2,0,0,1,1,0,0,2,0,4
4,M,25,2,6,1,0,2,5,5,5,3,0,5,3,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,M,25,1,997,4,5,0,0,0,4,2,4,0,2,0,0,0
9996,M,18,11,997,0,1,4,1,5,1,2,0,1,4,4,3,3
9997,F,25,12,997,2,3,0,2,5,1,4,5,0,3,3,3,0
9998,M,25,17,997,5,0,2,1,4,5,4,5,0,1,0,4,0


### Ba, Bbの読み込み

In [154]:
Ba = pd.read_csv(f"../data/B{ATTACK_TARGET}a.csv", names=ALL_HEADER)
Ba = Ba.drop(0)
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
1,Ilaire Garett,M,50,17,109,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Timofei Chastenet,M,25,16,945,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jule Sahnow,M,45,1,282,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Eldon Janusik,M,35,4,735,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Chuck Caren,M,45,17,927,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
Bb = pd.read_csv(f"../data/B{ATTACK_TARGET}b.csv")
Bb.head()

,2,56,247,260,653,673,810,885,1009,1073,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,0,4,0,4,0,3,0,0,2,*,...,0,4,5,1,0,0,2,5,2,4
1,1,0,1,5,2,2,2,2,0,5,...,0,1,0,0,0,1,0,0,0,5
2,1,0,0,4,3,4,3,2,0,0,...,2,3,0,1,0,5,0,1,3,3
3,2,4,2,5,5,5,0,0,*,0,...,5,2,5,2,3,1,3,3,2,0
4,0,0,1,5,1,1,0,1,5,5,...,0,4,0,1,2,0,0,1,0,0


## 分析

### GAOZに関して、BaのN番目の行と完全一致する行をC0~C9から取得　　
-GAOZへの変更が行われているか

In [156]:
# 試しにBaの3行目を選択
N = 3

gender = Ba.iloc[N]["Gender"]
age = int(Ba.iloc[N]["Age"])
occupation = int(Ba.iloc[N]["Occupation"])
zipcode = int(Ba.iloc[N]["ZIP-code"])

print("検索条件")
print(gender, age, occupation, zipcode)

検索条件
M 35 4 735


### GAOZが完全一致するものでなく、各Ciの中でHamming distが最小のものをリスト化する

In [157]:
import copy

def GAOZ_weighted_hamming(comp, i_gender, i_age, i_occupation, i_zipcode):
    ham = 0
    if comp['Gender'] != i_gender:
        ham += 1
    if comp['Age'] != i_age:
        ham += 2
    if  comp['Occupation'] != i_occupation:
        ham += 2
    if comp['ZIP-code'] != i_zipcode:
        ham += 4
    return ham

filtered_C_list = []
for c_index, cf in enumerate(c_data_list):
    min_hamming_distance = float("inf")
    cand = []
    # Cの10000レコードのそれぞれにハミング距離を計算
    for index, c_row in cf.iterrows():
        hamming_distance = GAOZ_weighted_hamming(c_row, gender, age, occupation, zipcode)
        if hamming_distance == min_hamming_distance:
            cand.append((c_row, hamming_distance))
        if hamming_distance < min_hamming_distance:
            min_hamming_distance = hamming_distance
            cand = [(c_row, hamming_distance)]
    # ハミング距離とCのファイルのindexを追加
    for elem, hamming_distance in cand:
        headers = copy.deepcopy(ALL_HEADER)
        headers.append("GAOZ_Hamming_Dist")
        headers.append("C_index")
        elem["GAOZ_Hamming_Dist"] = hamming_distance
        elem["C_index"] = c_index
        filtered_C_list.append(pd.Series(elem, index=headers))

merged_df = pd.DataFrame(filtered_C_list)
merged_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,GAOZ_Hamming_Dist,C_index
7043,NaN,F,56,4,735,NaN,NaN,NaN,4.0,0.0,...,4.0,2.0,0.0,NaN,NaN,NaN,0.0,3.0,3,0
7048,NaN,F,35,2,735,NaN,NaN,NaN,0.0,1.0,...,0.0,1.0,0.0,NaN,NaN,NaN,3.0,3.0,3,0
7054,NaN,M,25,4,735,5.0,0.0,NaN,1.0,3.0,...,NaN,NaN,NaN,NaN,5.0,5.0,0.0,4.0,2,1
7060,NaN,M,35,1,735,0.0,0.0,NaN,5.0,4.0,...,NaN,NaN,NaN,NaN,0.0,1.0,2.0,5.0,2,1
7052,NaN,M,25,4,735,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
7056,NaN,M,35,0,735,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
7064,NaN,M,35,17,735,0.0,0.0,NaN,NaN,NaN,...,0.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,2,3
7058,NaN,M,50,4,735,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2,4
7060,NaN,M,50,4,735,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,2,4
7033,NaN,M,35,9,735,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5


### データの再現　　
各行に対して最頻値を一つ値を取得する

In [158]:
import statistics
from collections import Counter

def select_random_mode(column):
    """
    最頻値を取得
    最頻値が複数ある場合はランダムに1つ選択
    """
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if not valid_values:
        # 有効な値がない場合はNaNを返す
        return np.nan
    
    # 最頻値とその出現回数を取得
    value_counts = Counter(valid_values)
    max_count = max(value_counts.values())
    
    # 最頻値のリストを作成
    modes = [value for value, count in value_counts.items() if count == max_count]
    
    # 最頻値が1つの場合はそれを返し、複数ある場合はランダムに1つ選択
    return np.random.choice(modes)
    
def select_random_value(column):
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if valid_values:
        # 有効な値がある場合、ランダムに1つ選択
        return np.random.choice(valid_values)
    else:
        # 有効な値がない場合はNaNを返す
        return np.nan

# 各カラムからランダムに値を選択
# random_values = {}
# for column in ALL_HEADER:
#     random_values[column] = select_random_value(merged_df[column])

# # 結果を新しいデータフレームとして作成
# result_df = pd.DataFrame([random_values])
# result_df

## 攻撃

### 以上の処理をBaの各行（0~49行）に対して実行する

In [159]:
import pandas as pd
import numpy as np

final_df = pd.DataFrame(columns=ALL_HEADER)

for row_index in range(50):
    # Baデータフレームから現在の行の情報を取得
    gender = Ba.iloc[row_index]["Gender"]
    age = int(Ba.iloc[row_index]["Age"])
    occupation = int(Ba.iloc[row_index]["Occupation"])
    zipcode = int(Ba.iloc[row_index]["ZIP-code"])

    filtered_C_list = []
    for c_index, cf in enumerate(c_data_list):
        min_hamming_distance = float("inf")
        cand = []
        # Cの10000レコードのそれぞれにハミング距離を計算
        for index, c_row in cf.iterrows():
            hamming_distance = GAOZ_weighted_hamming(c_row, gender, age, occupation, zipcode)
            if hamming_distance == min_hamming_distance:
                cand.append((c_row, hamming_distance))
            if hamming_distance < min_hamming_distance:
                min_hamming_distance = hamming_distance
                cand = [(c_row, hamming_distance)]
        # ハミング距離とCのファイルのindexを追加
        for elem, hamming_distance in cand:
            headers = copy.deepcopy(ALL_HEADER)
            headers.append("GAOZ_Hamming_Dist")
            headers.append("C_index")
            elem["GAOZ_Hamming_Dist"] = hamming_distance
            elem["C_index"] = c_index
            filtered_C_list.append(pd.Series(elem, index=headers))

    merged_df = pd.DataFrame(filtered_C_list)

    candidate_rows_df = pd.DataFrame(filtered_C_list)

    # 各列に対して、ランダムに値を選択（多数決の実装）
    elected_value = {}
    for column in ALL_HEADER:
        elected_value[column] = select_random_mode(candidate_rows_df[column])

    # 選択された値から新しい行を作成
    elected_value_row_df = pd.DataFrame([elected_value])

    # 選択された行をfinal_dfの最後に追加
    final_df = pd.concat([final_df, elected_value_row_df], ignore_index=True)

final_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,M,50,17,109,0.0,0.0,1.0,5.0,0.0,...,0.0,0.0,5.0,0.0,4.0,5.0,3.0,0.0,3.0,1.0
1,NaN,M,25,16,945,0.0,0.0,1.0,5.0,0.0,...,0.0,1.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
2,NaN,M,45,17,282,3.0,0.0,4.0,3.0,0.0,...,0.0,0.0,5.0,0.0,0.0,3.0,3.0,2.0,3.0,2.0
3,NaN,M,35,4,735,0.0,0.0,0.0,4.0,3.0,...,0.0,4.0,0.0,1.0,0.0,0.0,1.0,5.0,0.0,3.0
4,NaN,M,45,17,927,2.0,0.0,1.0,5.0,0.0,...,0.0,3.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0
5,NaN,M,35,5,26,0.0,0.0,0.0,5.0,3.0,...,0.0,0.0,2.0,0.0,0.0,3.0,3.0,3.0,5.0,0.0
6,NaN,M,25,15,880,0.0,0.0,3.0,5.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0
7,NaN,F,18,4,939,4.0,0.0,1.0,5.0,2.0,...,0.0,0.0,4.0,3.0,0.0,0.0,2.0,3.0,0.0,5.0
8,NaN,M,56,11,45,3.0,0.0,0.0,4.0,5.0,...,0.0,0.0,0.0,3.0,0.0,3.0,2.0,0.0,0.0,0.0
9,NaN,F,35,7,282,0.0,0.0,0.0,4.0,0.0,...,3.0,5.0,1.0,3.0,1.0,5.0,3.0,3.0,1.0,0.0


### final_dfをもとに、Bbのそれぞれの行に対応しそうなBaのインデックスと穴埋めの値を出力

In [160]:
answer_list = []
for index, row in Bb.iterrows():
    answer_index = 0
    answer_value = 0
    min_ham = float('inf')
    for cand_index, cand_row in final_df.iterrows():
        ham = 0
        for id in MOVIE_IDS:
            if row[id] != '*':
                if (int(row[id]) != int(cand_row[id])):
                    ham += 1
        if ham < min_ham:
            min_ham = ham
            answer_index = cand_index
    # print(f'{len(row)}')
    for id in MOVIE_IDS:
        if row[id] == '*':
            answer_value = int(final_df.iloc[answer_index][id])
            break
    # print(f'{answer_index}, ')
    answer_list.append([answer_index, answer_value])
print(answer_list)

[[15, 0], [20, 0], [6, 0], [18, 2], [44, 0], [16, 0], [38, 3], [7, 4], [21, 0], [15, 0], [18, 5], [34, 5], [22, 4], [8, 0], [43, 0], [10, 1], [9, 0], [15, 0], [39, 5], [10, 0], [9, 0], [17, 5], [13, 5], [45, 0], [38, 0], [9, 4], [15, 3], [34, 0], [44, 1], [6, 0], [11, 0], [15, 0], [20, 0], [15, 5], [15, 0], [20, 4], [38, 0], [44, 1], [39, 0], [17, 0], [39, 0], [4, 5], [30, 4], [2, 3], [37, 2], [15, 3], [34, 0], [33, 0], [43, 1], [46, 2]]


### 最終的に得た映画のレビューの候補とBbを比較し、Bbの何番目のindexが一番近くてハミング距離にどれくらいの違いがあるかを取得する

In [161]:
def hamming_distance(a, b):
    a_int = a.fillna(0).astype(int)
    b_int = b.replace('*', None).fillna(0).astype(int)
    # ハミング距離の計算
    return np.sum(a_int != b_int)

def find_closest_row(row, Bb):
    distances = Bb.apply(lambda bb_row: hamming_distance(row[MOVIE_IDS], bb_row[MOVIE_IDS]), axis=1)
    min_distance = distances.min()
    closest_index = distances.idxmin()
    return pd.Series({'Min_Hamming_Distance': min_distance, 'Closest_Bb_Index': closest_index})

# 結果を格納するデータフレーム
result_df = pd.DataFrame(index=final_df.index, columns=['Min_Hamming_Distance', 'Closest_Bb_Index'])

# 各行に対してハミング距離を計算し、最も近い行を見つける
for idx, row in final_df[MOVIE_IDS].iterrows():
    result = find_closest_row(row, Bb[MOVIE_IDS])
    result_df.loc[idx] = result

# 元のfinal_dfにハミング距離の結果を結合
final_result_df = pd.concat([final_df, result_df], axis=1)
final_result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,Min_Hamming_Distance,Closest_Bb_Index
0,NaN,M,50,17,109,0.0,0.0,1.0,5.0,0.0,...,5.0,0.0,4.0,5.0,3.0,0.0,3.0,1.0,26,17
1,NaN,M,25,16,945,0.0,0.0,1.0,5.0,0.0,...,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,22,17
2,NaN,M,45,17,282,3.0,0.0,4.0,3.0,0.0,...,5.0,0.0,0.0,3.0,3.0,2.0,3.0,2.0,29,1
3,NaN,M,35,4,735,0.0,0.0,0.0,4.0,3.0,...,0.0,1.0,0.0,0.0,1.0,5.0,0.0,3.0,23,26
4,NaN,M,45,17,927,2.0,0.0,1.0,5.0,0.0,...,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,18,9
5,NaN,M,35,5,26,0.0,0.0,0.0,5.0,3.0,...,2.0,0.0,0.0,3.0,3.0,3.0,5.0,0.0,21,28
6,NaN,M,25,15,880,0.0,0.0,3.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,22,24
7,NaN,F,18,4,939,4.0,0.0,1.0,5.0,2.0,...,4.0,3.0,0.0,0.0,2.0,3.0,0.0,5.0,26,17
8,NaN,M,56,11,45,3.0,0.0,0.0,4.0,5.0,...,0.0,3.0,0.0,3.0,2.0,0.0,0.0,0.0,18,9
9,NaN,F,35,7,282,0.0,0.0,0.0,4.0,0.0,...,1.0,3.0,1.0,5.0,3.0,3.0,1.0,0.0,25,24


In [162]:

# 結果を格納するデータフレーム
result_df = pd.DataFrame(index=final_df.index, columns=['Min_Hamming_Distance', 'Closest_Bb_Index'])

# 各行に対してハミング距離を計算し、最も近い行を見つける
for idx, row in final_df[MOVIE_IDS].iterrows():
    result = find_closest_row(row, Bb[MOVIE_IDS])
    result_df.loc[idx] = result

# 元のfinal_dfにハミング距離の結果を結合
final_result_df = pd.concat([final_df, result_df], axis=1)
final_result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,Min_Hamming_Distance,Closest_Bb_Index
0,NaN,M,50,17,109,0.0,0.0,1.0,5.0,0.0,...,5.0,0.0,4.0,5.0,3.0,0.0,3.0,1.0,26,17
1,NaN,M,25,16,945,0.0,0.0,1.0,5.0,0.0,...,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,22,17
2,NaN,M,45,17,282,3.0,0.0,4.0,3.0,0.0,...,5.0,0.0,0.0,3.0,3.0,2.0,3.0,2.0,29,1
3,NaN,M,35,4,735,0.0,0.0,0.0,4.0,3.0,...,0.0,1.0,0.0,0.0,1.0,5.0,0.0,3.0,23,26
4,NaN,M,45,17,927,2.0,0.0,1.0,5.0,0.0,...,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,18,9
5,NaN,M,35,5,26,0.0,0.0,0.0,5.0,3.0,...,2.0,0.0,0.0,3.0,3.0,3.0,5.0,0.0,21,28
6,NaN,M,25,15,880,0.0,0.0,3.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,22,24
7,NaN,F,18,4,939,4.0,0.0,1.0,5.0,2.0,...,4.0,3.0,0.0,0.0,2.0,3.0,0.0,5.0,26,17
8,NaN,M,56,11,45,3.0,0.0,0.0,4.0,5.0,...,0.0,3.0,0.0,3.0,2.0,0.0,0.0,0.0,18,9
9,NaN,F,35,7,282,0.0,0.0,0.0,4.0,0.0,...,1.0,3.0,1.0,5.0,3.0,3.0,1.0,0.0,25,24
